In [1]:
import re
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv1D
from keras.optimizers import Adamax

import numpy as np
import random
import sys
import io
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
ghazals=pd.read_excel('urdu/faiz/ghazals.xlsx')
ghazals=ghazals.reset_index()
del ghazals['index']
ghazals

In [3]:
#Because i want it to be purly urdu so i'll delete all the nazam's with english

ghazals_pruned=pd.DataFrame(columns=list(['title','text']))
for counter,i in enumerate(ghazals['text']):
    if(re.search('[a-zA-Z]', i))==None:
        df = pd.DataFrame([[ghazals['title'][counter], i]],columns=list(['title','text']))
        ghazals_pruned=ghazals_pruned.append(df, ignore_index=True)

In [4]:
#Finding the longest line in literature
longest=0
for i in ghazals_pruned['text']:
    #Finding the longest line in the literature
    for j in i.split('\n'):
        if longest<len(j):
            longest=len(j)
print("Longest line in the literature",longest)

Longest line in the literature 63


In [5]:
text=''
for i in ghazals_pruned['text']:
    #Creating the dataset
    text=text+i
print("Length of the literature:",len(text))

Length of the literature: 36245


In [6]:
new_text=''
for i in ghazals_pruned['text']:
    #Finding the longest line in the literature
    for j in i.split('\n'):
        if(len(j)<=2):
            continue
        j=j.strip()
        j=j.ljust(longest-1, ' ')
        j=j+'\n'
        new_text=new_text+j
ghazals_final=new_text

In [7]:
ghazals_final = ghazals_final.replace("'", "")

In [ ]:
nazams=pd.read_excel('urdu/faiz/nazams.xlsx')
nazams=nazams.reset_index()
del nazams['index']
nazams

In [9]:
#Because i want it to be purly urdu so i'll delete all the nazam's with english

nazams_pruned=pd.DataFrame(columns=list(['title','text']))
for counter,i in enumerate(nazams['text']):
    if(re.search('[a-zA-Z]', i))==None:
        df = pd.DataFrame([[nazams['title'][counter], i]],columns=list(['title','text']))
        nazams_pruned=nazams_pruned.append(df, ignore_index=True)

In [ ]:
nazams_pruned

In [11]:
#Finding the longest line in literature
# longest=0
# for i in nazams_pruned['text']:
#     #Finding the longest line in the literature
#     for j in i.split('\n'):
#         if longest<len(j):
#             longest=len(j)
# print("Longest line in the literature",longest)

In [12]:
text=''
for i in nazams_pruned['text']:
    #Creating the dataset
    text=text+i
print("Length of the literature:",len(text))

Length of the literature: 52424


In [13]:
new_text=''
for i in nazams_pruned['text']:
    #Finding the longest line in the literature
    for j in i.split('\n'):
        if(len(j)<=2):
            continue
        j=j.strip()
        j=j.ljust(longest-1, ' ')
        j=j+'\n'
        new_text=new_text+j
nazams_final=new_text

In [14]:
nazams_final = nazams_final.replace("'", "")

In [15]:
nazams_final = nazams_final.replace("(1)", "")
nazams_final = nazams_final.replace("(۲)", "")

In [ ]:
for i in nazams_final.split('\n'):
    print(i)

In [17]:
#Finding unique characters
chars = sorted(list(set(nazams_final+ghazals_final)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 52


In [18]:
chars

['\n',
 ' ',
 '!',
 '،',
 'ؔ',
 '؟',
 'ء',
 'آ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'و',
 'ٓ',
 'ٔ',
 'ٰ',
 'ٹ',
 'پ',
 'چ',
 'ڈ',
 'ڑ',
 'ک',
 'گ',
 'ں',
 'ھ',
 'ہ',
 'ۂ',
 'ی',
 'ے',
 '۔']

In [19]:
len(ghazals_final+nazams_final)

169462

In [20]:
final_text=ghazals_final+nazams_final

In [21]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = longest
step = longest
sentences = []
next_chars = []

for i in range(0, len(final_text) - maxlen, step+step):
    sentences.append(final_text[i: i + maxlen])
    next_chars.append(final_text[i + maxlen:i+maxlen + maxlen])

print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        
        
for i, sentence in enumerate(next_chars):    
    for t,char in enumerate(sentence):
        y[i,t, char_indices[char]] = 1



nb sequences: 1345
Vectorization...


In [22]:
print(x[0].astype(int))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [23]:
print(y[0].astype(int))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [24]:
x.shape

(1345, 63, 52)

In [25]:
y.shape

(1345, 63, 52)

In [26]:
dense_dim=y.shape[1]*y.shape[2]


In [28]:
from keras.layers import Reshape,Dropout
from keras.callbacks import ModelCheckpoint
#Model callbacks
filepath="faiz--cnn-256-{val_loss:.5f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]


In [39]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(Conv1D(256,kernel_size=2,padding='same',input_shape=(maxlen, len(chars))))
model.add(Dropout(0.3))
model.add(Conv1D(256,kernel_size=2,padding='same'))
model.add(Dropout(0.3))
model.add(Conv1D(256,kernel_size=2,padding='same'))
model.add(Dropout(0.3))
model.add(Conv1D(52,kernel_size=2,padding='same'))

# model.add(Reshape((maxlen, len(chars))))
model.add(Activation('sigmoid'))
optimizer = Adamax(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, 63, 256)           26880     
_________________________________________________________________
dropout_13 (Dropout)         (None, 63, 256)           0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 63, 256)           131328    
_________________________________________________________________
dropout_14 (Dropout)         (None, 63, 256)           0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 63, 256)           131328    
_________________________________________________________________
dropout_15 (Dropout)         (None, 63, 256)           0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 63, 52)            26676 

In [40]:
history=model.fit(x, y,
          batch_size=64,
          epochs=100,
          validation_split=0.2,callbacks=callbacks_list)

Train on 1076 samples, validate on 269 samples
Epoch 1/100
1076/1076 [==============================] - 4s 4ms/step - loss: 0.1725 - val_loss: 0.0818
Epoch 2/100
1076/1076 [==============================] - 0s 432us/step - loss: 0.0603 - val_loss: 0.0504
Epoch 3/100
1076/1076 [==============================] - 0s 379us/step - loss: 0.0491 - val_loss: 0.0463
Epoch 4/100
1076/1076 [==============================] - 0s 388us/step - loss: 0.0468 - val_loss: 0.0448
Epoch 5/100
1076/1076 [==============================] - 0s 386us/step - loss: 0.0456 - val_loss: 0.0438
Epoch 6/100
1076/1076 [==============================] - 0s 405us/step - loss: 0.0448 - val_loss: 0.0434
Epoch 7/100
1076/1076 [==============================] - 0s 428us/step - loss: 0.0444 - val_loss: 0.0432
Epoch 8/100
1076/1076 [==============================] - 1s 487us/step - loss: 0.0442 - val_loss: 0.0430
Epoch 9/100
1076/1076 [==============================] - 1s 517us/step - loss: 0.0440 - val_loss: 0.0429
Epoch 10/1

Epoch 34/100
1076/1076 [==============================] - 0s 389us/step - loss: 0.0427 - val_loss: 0.0425
Epoch 35/100
1076/1076 [==============================] - 0s 414us/step - loss: 0.0427 - val_loss: 0.0425
Epoch 36/100
1076/1076 [==============================] - 0s 354us/step - loss: 0.0427 - val_loss: 0.0425
Epoch 37/100
1076/1076 [==============================] - 0s 370us/step - loss: 0.0427 - val_loss: 0.0426
Epoch 38/100
1076/1076 [==============================] - 0s 406us/step - loss: 0.0426 - val_loss: 0.0426
Epoch 39/100
1076/1076 [==============================] - 0s 408us/step - loss: 0.0426 - val_loss: 0.0426
Epoch 40/100
1076/1076 [==============================] - 0s 375us/step - loss: 0.0426 - val_loss: 0.0425
Epoch 41/100
1076/1076 [==============================] - 0s 379us/step - loss: 0.0426 - val_loss: 0.0426
Epoch 42/100
1076/1076 [==============================] - 0s 379us/step - loss: 0.0426 - val_loss: 0.0426
Epoch 43/100
1076/1076 [======================

Epoch 73/100
1076/1076 [==============================] - 0s 378us/step - loss: 0.0421 - val_loss: 0.0426
Epoch 74/100
1076/1076 [==============================] - 0s 381us/step - loss: 0.0422 - val_loss: 0.0427
Epoch 75/100
1076/1076 [==============================] - 0s 375us/step - loss: 0.0422 - val_loss: 0.0427
Epoch 76/100
1076/1076 [==============================] - 0s 387us/step - loss: 0.0421 - val_loss: 0.0426
Epoch 77/100
1076/1076 [==============================] - 0s 398us/step - loss: 0.0421 - val_loss: 0.0426
Epoch 78/100
1076/1076 [==============================] - 0s 376us/step - loss: 0.0421 - val_loss: 0.0427
Epoch 79/100
1076/1076 [==============================] - 0s 386us/step - loss: 0.0421 - val_loss: 0.0426
Epoch 80/100
1076/1076 [==============================] - 0s 375us/step - loss: 0.0422 - val_loss: 0.0429
Epoch 81/100
1076/1076 [==============================] - 0s 384us/step - loss: 0.0421 - val_loss: 0.0426
Epoch 82/100
1076/1076 [======================

In [28]:
model.load_weights('faiz-gru-0.03959.hdf5')

In [41]:
pred=model.predict(x)

In [42]:
a=x[10]
b=pred[10]

In [43]:
b

array([[3.59821890e-04, 1.34702191e-01, 3.48691974e-04, ...,
        8.18703175e-02, 2.74913423e-02, 3.35998309e-04],
       [2.45026051e-04, 1.11425154e-01, 2.57188571e-04, ...,
        1.37302637e-01, 5.62129803e-02, 2.39955101e-04],
       [2.17079054e-04, 2.09057882e-01, 1.87180121e-04, ...,
        7.93236718e-02, 3.66433896e-02, 1.43109966e-04],
       ...,
       [2.11088845e-04, 3.41324329e-01, 4.37593146e-04, ...,
        3.75167020e-02, 2.05364134e-02, 1.00227204e-04],
       [1.67911101e-04, 5.67973554e-01, 8.77263228e-05, ...,
        3.81039307e-02, 3.74093726e-02, 1.42378067e-05],
       [1.26771466e-03, 7.09475219e-01, 4.37989016e-04, ...,
        3.21452394e-02, 1.98939200e-02, 2.52456899e-04]], dtype=float32)

In [44]:
a=(a == a.max(axis=1, keepdims=True)).astype(int)
b=(b == b.max(axis=1, keepdims=True)).astype(int)

In [45]:
b

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [46]:
a=a.argmax(axis=1)
b=b.argmax(axis=1)

In [47]:
a

array([14, 10, 33, 10, 33, 47,  1, 31, 49, 50,  1, 32, 21, 12, 49, 36,  1,
       39, 49, 32, 10, 33, 47,  1, 31, 49, 50,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  0, 15, 32, 17,  1], dtype=int64)

In [48]:
b

array([10, 10,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, 10,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1], dtype=int64)

In [49]:
text=''
for i in a:
    text=text+indices_char[i]
print(text)
text=''
for i in b:
    text=text+indices_char[i]
print(text)

جانانہ لیے مستیٔ پیمانہ لیے                               
حمد 
اا                ا                                       
    
